In [10]:
from scipy.optimize         import newton
from sklearn.neighbors      import KDTree
from scipy.sparse           import csr_matrix
from scipy.sparse.csgraph   import dijkstra
from scipy.linalg           import eig
from sklearn.decomposition  import PCA
import numpy                as np
import gudhi                as gd
import warnings
import gurobipy             as gp
from gurobipy               import GRB
import matplotlib.pyplot    as plt
from kneed                  import KneeLocator
from scipy.spatial          import distance_matrix
from ripser                 import Rips
from sklearn.preprocessing  import PolynomialFeatures
import sympy                as sp
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem     import Problem
from pymoo.optimize         import minimize
import tadasets
import rml
from scipy.spatial import Delaunay

#%matplotlib qt

In [3]:
n_points = 1500

In [60]:
params = {'max_components':5, 'S':0.5, 'k':10, 'threshold_var':0.05, 'edge_sen':1.5, 'k0':10, 'beta':[0.8, 3, 0.2, 0.2]}
#params = {'max_components':5, 'S':0.5, 'k':10, 'threshold_var':0.05, 'edge_sen':2, 'k0':3, 'beta':None}

b_params = {'k':10, 'threshold_var':0.08, 'edge_sen':1, 'k0':10} 

In [37]:
sphere1 = tadasets.dsphere(n=int(n_points))

sphere5 = tadasets.dsphere(n=int(n_points*1.3))
sphere5 = sphere5[sphere5[:, 2]>=-0.7]

In [38]:
S = rml.Simplex()
S.build_simplex(sphere1, **params)
_ = S.normal_coords_trade(**params)

388
44


In [39]:
S_b = rml.Simplex()
S_b.build_simplex(S.coords, **b_params)

In [41]:
fig = plt.figure(figsize=(20, 20))
fig.suptitle(", ".join([i+"="+str(params[i]) for i in params.keys()]) + f', dim={S.dim}, n={n_points}')

ax1 = fig.add_subplot(1, 3, 1)
ax2 = fig.add_subplot(1, 3, 2)
ax3 = fig.add_subplot(1, 3, 3)

ax1.scatter(S.coords[:, 0], S.coords[:, 1], c=S.pointcloud[:, 0])

tri = Delaunay(S.coords)
boundary = set()
for i in range(len(tri.neighbors)):
    for k in range(3):
        if (tri.neighbors[i][k] == -1):
            nk1,nk2 = (k+1)%3, (k+2)%3 
            boundary.add(tri.simplices[i][nk1])
            boundary.add(tri.simplices[i][nk2])

ax2.triplot(S.coords[:,0], S.coords[:,1], tri.simplices.copy())
ax2.plot(S.coords[list(boundary),0], S.coords[list(boundary),1], 'or')
#plt.scatter(S.coords[:,0], S.coords[:,1], c=S.pointcloud[:, 0])

ax3.scatter(S.coords[:, 0], S.coords[:, 1], c=S.pointcloud[:, 0])

for i in range(len(S_b.pointcloud)):
    for k in S_b.edges[i]:
        ax3.plot([S_b.pointcloud[i][0], S_b.pointcloud[k][0]],[S_b.pointcloud[i][1], S_b.pointcloud[k][1]], color='black', alpha=0.1)

plt.show()

In [57]:
def generate_disk(n, r=1):
    U1 = np.random.uniform(size = 10000)
    U2 = np.random.uniform(size = 10000)
    X = r * np.sqrt(U2) * np.cos(2 * np.pi * U1)
    Y = r * np.sqrt(U2) * np.sin(2 * np.pi * U1)

    return X, Y

In [59]:
X, Y = generate_disk(1000)
disk_data = np.column_stack([X, Y])
annulus = disk_data[np.linalg.norm(disk_data, axis=1)>=0.5]
plt.scatter(annulus[:, 0], annulus[:, 1])

In [61]:
S_c = rml.Simplex()
S_b.build_simplex(annulus, **b_params)
p_idx, _ = S.normal_coords_trade(**b_params)